In [1]:
# import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [6]:
import pandas as pd
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

# convert from pandas dataframe to tensor
data = torch.tensor( iris[iris.columns[0:4]].values ).float()

# transform species to number
labels = torch.zeros(len(data), dtype=torch.long)
# labels[iris.species=='setosa'] = 0 # don't need!
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

In [2]:
class ANNiris(nn.Module):
    
    def __init__(self, nNodes, nLayers):
        super().__init__()
        
        self.layers = nn.ModuleDict() # dict to store layers
        self.nLayers = nLayers
        
        # input layer
        self.layers['input'] = nn.Linear(4, nNodes) # 4 input variables
        
        for i in range(nLayers):
            self.layers[f'hidden-L{i + 1}'] = nn.Linear(nNodes, nNodes)   
            
        # output layer
        self.layers['output'] = nn.Linear(nNodes, 3) # 3 output categories
    
    def forward(self, x):
        x = F.relu(self.layers['input'](x))
        
        for i in range(self.nLayers):
            x = F.relu(self.layers[f'hidden-L{i + 1}'](x))
        
        x = self.layers['output'](x)
        return x

In [ ]:
numEpochs = 200

def trainModel(model):
    lossFn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.paramters(), lr=0.1)
    
    for epch in range(numEpochs):
        
        # forward pass
        yPred = model(data)
        
        # compute loss
        loss = lossFn(yPred, labels)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # final forward pass
    predictions = model(data)
    predLabels = torch.argmax(predictions, axis=1)
    acc = 100 * torch.mean((predLabels == labels).float())
    
    # total number of trainable params in model
    nParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    return acc, nParams
        